# TasteNet-MNL 

El presente notebook busca replicar experimentos de los autores del paper de TasteNet-MNL con el uso de la librería Choice Learn de python.

Ventajas de utilizar la implementación de Choice Learn por sobre las implementaciones de los autores de los modelos:

- Escalabilidad

- Enfoque a los grandes volúmenes de datos

- Mantenimiento 

- Documentación

- Generalizacion